<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">


# Predicting Shots Made Per Game by Kobe Bryant

_Authors: Kiefer Katovich (SF)_

---

In this lab you'll be using regularized regression penalties — ridge, lasso, and elastic net — to try and predict how many shots Kobe Bryant made per game during his career.

The Kobe Shots data set contains hundreds of columns representing different characteristics of each basketball game. Fitting an ordinary linear regression using every predictor would dramatically overfit the model, considering the limited number of observations (games) we have available. Plus, many of the predictors have significant multicollinearity. 


**Warning:** Some of these calculations are computationally expensive and may take a while to execute. It may be worthwhile to only use a portion of the data to perform these calculations, especially if you've experienced kernel issues in the past.

---

### 1) Load packages and data.

In [29]:
import numpy as np
import pandas as pd
import patsy

from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [20]:
kobe = pd.read_csv('./datasets/kobe_superwide_games.csv')

---

### 2) Examine the data.

- How many columns are there?  645
- Examine what the observations (rows) and columns represent.
- Why might regularization be particularly useful for modeling this data?

2.1: 645

2.2: Each row represents a game played by Kobe Bryant.  The columns show the season in which a game took place against a certain team.  There are also columns specifying what kind of shot exactly was made.

2.3:  Regularization could be particularly helpful for modeling this data because there are so many features that can be input and we would appreciate some help reducing the features that contain a lot of 'noise'.

In [21]:
kobe.shape

(1558, 645)

In [22]:
kobe.head()

,SHOTS_MADE,AWAY_GAME,SEASON_OPPONENT:atl:1996-97,SEASON_OPPONENT:atl:1997-98,SEASON_OPPONENT:atl:1999-00,SEASON_OPPONENT:atl:2000-01,SEASON_OPPONENT:atl:2001-02,SEASON_OPPONENT:atl:2002-03,SEASON_OPPONENT:atl:2003-04,SEASON_OPPONENT:atl:2004-05,...,ACTION_TYPE:tip_layup_shot,ACTION_TYPE:tip_shot,ACTION_TYPE:turnaround_bank_shot,ACTION_TYPE:turnaround_fadeaway_bank_jump_shot,ACTION_TYPE:turnaround_fadeaway_shot,ACTION_TYPE:turnaround_finger_roll_shot,ACTION_TYPE:turnaround_hook_shot,ACTION_TYPE:turnaround_jump_shot,SEASON_GAME_NUMBER,CAREER_GAME_NUMBER
0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
1,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2
2,2.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3
3,2.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,4
4,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,5


In [23]:
kobe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1558 entries, 0 to 1557
Columns: 645 entries, SHOTS_MADE to CAREER_GAME_NUMBER
dtypes: float64(640), int64(5)
memory usage: 7.7 MB


---

### 3) Create predictor and target variables. Standardize the predictors.

Why is normalization necessary for regularized regressions?

Use the `sklearn.preprocessing` class `StandardScaler` to standardize the predictors.

# A:

Normalizing the data is necessary for regularized regressions because our different inputs may/are often measured on different scales, which will wrongly influence the Beta Coefficients of our features.  In other words, we normalize to ensure that all inputs are measured on a similar scale so that their coefficients are weighed on a similar scale as well. 

In [26]:
numerical_cols = kobe._get_numeric_data()

In [27]:
features = [col for col in numerical_cols if col != 'SHOTS_MADE']

In [28]:
X = kobe[features]
y = kobe['SHOTS_MADE']

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [31]:
ss = StandardScaler()

In [32]:
ss.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [33]:
X_train_sc = ss.transform(X_train)
X_test_sc = ss.transform(X_test)

---

### 4. Build a linear regression predicting `SHOTS_MADE` from the rest of the columns.

Cross-validate the $R^2$ of an ordinary linear regression model with 10 cross-validation folds.

How does it perform?

# A:  

#### The linear regression model performs very poorly, as can be seen by the mean cross val score in the negatives.

In [34]:
lr = LinearRegression()

In [35]:
cross_val_score(lr, X_train_sc, y_train, cv=10).mean()

-1.9299028749702666e+28

---

### 5) Find an optimal value for the ridge regression alpha using `RidgeCV`.

Go to the documentation and [read how RidgeCV works](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).

> *Hint: Once the RidgeCV is fit, the attribute `.alpha_` contains the best alpha parameter it found through cross-validation.*

Recall that ridge performs best when searching alphas through logarithmic space (`np.logspace`). This may take awhile to fit.


In [36]:
ridge = RidgeCV()

In [46]:
ridge.fit(X_train_sc, y_train)

RidgeCV(alphas=(0.1, 1.0, 10.0), cv=None, fit_intercept=True, gcv_mode=None,
    normalize=False, scoring=None, store_cv_values=False)

In [47]:
ridge.alpha_

10.0

---

### 6) Cross-validate the ridge regression $R^2$ with the optimal alpha.

Is it better than the linear regression? If so, why might this be?

# A:

####  Yes, it is better than the linear regression.  This may be because the ridge regression has limited the effects of 'non-important' features, or noise, when it created its model.

In [48]:
cross_val_score(ridge, X_train_sc, y_train, cv=10).mean()

0.40269539415566796

---

### 7) Find an optimal value for lasso regression alpha using `LassoCV`.

Go to the documentation and [read how LassoCV works](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html). It is very similar to `RidgeCV`.

> *Hint: Again, once the `LassoCV` is fit, the attribute `.alpha_` contains the best alpha parameter it found through cross-validation.*

Recall that lasso, unlike ridge, performs best when searching for alpha through linear space (`np.linspace`). However, you can actually let the LassoCV decide what alphas to use itself by setting the keyword argument `n_alphas=` to however many alphas you want it to search over. We recommend letting scikit-learn choose the range of alphas.

_**Tip:** If you find your CV taking a long time and you're not sure if it's working, set `verbose =1`._

In [49]:
lasso = LassoCV()

In [50]:
lasso.fit(X_train_sc, y_train)

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of

LassoCV(alphas=None, copy_X=True, cv=None, eps=0.001, fit_intercept=True,
    max_iter=1000, n_alphas=100, n_jobs=1, normalize=False, positive=False,
    precompute='auto', random_state=None, selection='cyclic', tol=0.0001,
    verbose=False)

In [51]:
lasso.alpha_

0.10641451929726059

---

### 8) Cross-validate the lasso $R^2$ with the optimal alpha.

Is it better than the linear regression? Is it better than ridge? What do the differences in results imply about the issues with the data set?

# A:

#### Yes, the lasso regression model works better than both the linear regression and the ridge.  The differences in results implies that the data set contains too many columns or predictors that are noise and do not truly impact Kobe Bryant's amount of shots made.

In [52]:
cross_val_score(lasso, X_train_sc, y_train, cv=10).mean()

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of

0.6313409992429427

---

### 9) Look at the coefficients for variables in the lasso.

1. Show the coefficient for variables, ordered from largest to smallest coefficient by absolute value.
2. What percent of the variables in the original data set are "zeroed-out" by the lasso?
3. What are the most important predictors for how many shots Kobe made in a game?

> **Note:** If you only fit the lasso within `cross_val_score`, you'll have to refit it outside of that function to pull out the coefficients.

In [ ]:
lasso.fit(X_train_sc, y_train)

In [94]:
col_list = list(kobe.columns)

In [96]:
col_list.remove('SHOTS_MADE')

In [102]:
coef_list = list(lasso.coef_)

In [107]:
lasso_dict = dict(zip(col_list, coef_list))

In [134]:
absolute_lasso_dict = {}

for k, v in lasso_dict.items():
    absolute_lasso_dict[k] = abs(v)

### 1. Show the coefficient for variables, ordered from largest to smallest coefficient by absolute value.

#### Please see below the variables and their absolute-value-coefficients, from largest to smallest.

In [137]:
import operator

sorted(absolute_lasso_dict.items(), key=operator.itemgetter(1), reverse=True)

[('COMBINED_SHOT_TYPE:jump_shot', 1.45701649195532),
 ('SHOT_TYPE:2pt_field_goal', 0.6179224411893001),
 ('SHOT_ZONE_BASIC:restricted_area', 0.4825784448251801),
 ('COMBINED_SHOT_TYPE:dunk', 0.3636053042436625),
 ('ACTION_TYPE:jump_shot', 0.2616596028383961),
 ('SECONDS_REMAINING', 0.17853307265205004),
 ('SHOT_ZONE_AREA:right_side_center(rc)', 0.16367594877379987),
 ('SHOT_ZONE_AREA:center(c)', 0.10401419760737111),
 ('SEASON_OPPONENT:nyk:2008-09', 0.10221714891416465),
 ('SEASON_OPPONENT:mem:2009-10', 0.10172125157749642),
 ('SEASON_OPPONENT:sea:2004-05', 0.09424102501194119),
 ('SEASON_OPPONENT:bos:2015-16', 0.09174436947166077),
 ('ACTION_TYPE:layup_shot', 0.0907653545881624),
 ('SEASON_OPPONENT:sac:1999-00', 0.09003236233275837),
 ('SEASON_OPPONENT:sas:2014-15', 0.0632199527329714),
 ('SEASON_OPPONENT:phx:2007-08', 0.06105658873051661),
 ('SEASON_OPPONENT:phx:2011-12', 0.054788647624500655),
 ('SEASON_OPPONENT:mem:2004-05', 0.05021294288160597),
 ('SEASON_OPPONENT:dal:2004-05', 0.

### 2. What percent of the variables in the original data set are "zeroed-out" by the lasso?
#### About 90% of the variables in the original data set were zeroed-out by the lasso.

In [148]:
percent_zero = 100 * (coef_list.count(0)/len(col_list))
percent_zero

90.21739130434783

### 3. What are the most important predictors for how many shots Kobe made in a game?

#### Please find below the top 10 most important predictors for how many shots Kobe made in a game:

In [154]:
non_zero_lasso_dict = {}

for k, v in absolute_lasso_dict.items():
    if v != 0:
        non_zero_lasso_dict[k] = v
    else:
        pass
    
sorted(non_zero_lasso_dict.items(), key=operator.itemgetter(1), reverse=True)[0:10]

[('COMBINED_SHOT_TYPE:jump_shot', 1.45701649195532),
 ('SHOT_TYPE:2pt_field_goal', 0.6179224411893001),
 ('SHOT_ZONE_BASIC:restricted_area', 0.4825784448251801),
 ('COMBINED_SHOT_TYPE:dunk', 0.3636053042436625),
 ('ACTION_TYPE:jump_shot', 0.2616596028383961),
 ('SECONDS_REMAINING', 0.17853307265205004),
 ('SHOT_ZONE_AREA:right_side_center(rc)', 0.16367594877379987),
 ('SHOT_ZONE_AREA:center(c)', 0.10401419760737111),
 ('SEASON_OPPONENT:nyk:2008-09', 0.10221714891416465),
 ('SEASON_OPPONENT:mem:2009-10', 0.10172125157749642)]

---

### 10) Find an optimal value for elastic net regression alpha using `ElasticNetCV`.

Go to the documentation and [read how ElasticNetCV works](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNetCV.html).

Note that here you'll be optimizing both the alpha parameter and the l1_ratio:
- `alpha`: Strength of regularization.
- `l1_ratio`: Amount of ridge vs. lasso (0 = all ridge, 1 = all lasso).
    
Do not include 0 in the search for `l1_ratio` — it won't allow it and will break.

You can use `n_alphas` for the alpha parameters instead of setting your own values, which we highly recommend.

Also, be careful setting too many l1_ratios over cross-validation folds in your search. It can take a long time if you choose too many combinations and, for the most part, there are diminishing returns in this data.

In [ ]:
# A:

In [156]:
enet_alphas = np.arange(0.5, 1.0, 0.005)

In [157]:
enet_ratio = 0.5

In [168]:
enet_model = ElasticNetCV(alphas=enet_alphas, l1_ratio=enet_ratio, cv=5)

In [169]:
enet_model = enet_model.fit(X_train_sc, y_train)

In [170]:
enet_model.alpha_

0.5

---

### 11) Cross-validate the elastic net $R^2$ with the optimal alpha and l1_ratio.

How does it compare to the ridge and lasso regularized regressions?

# A:

#### It seems to work better than Ridge but worse than Lasso.

In [171]:
cross_val_score(enet_model, X_train_sc, y_train, cv=10).mean()

0.6034197199108567

---

### 12. [Bonus] Compare the residuals for ridge and lasso visually.


In [ ]:
# A: Maybe a jointplot?